<H1>Shark Attack Data Cleaning</H1>
In this nootebook we clean all the important variables and mapped to an identifier, next generate a .csv file with the cleaned variables.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import sys
import re
from random import randint

reload(sys)
sys.setdefaultencoding('utf-8')

In [73]:
data = pd.read_csv("./attacks.csv", encoding = "ISO-8859-1",header=0)
#Clean corrupted data
data.drop(data.index[[60,272,273,1566,2063,2126,2376,2457,4303,4530,4805]])
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
dtype: object

<h1>Date Cleaning</h1>
At the moment the dates with a different length of 10 chars are invalidated

In [3]:
def extract_date(date):
    str_date= str(date)
    str_date = str.replace(str_date,'.','/')
    str_date = str.replace(str_date,'-','/')
    str_date = str_date[:10]
    
    if len(str_date)==10:        
        return str_date
    else:
        return "0000/00/01"

In [4]:
data['Clean Date'] = data['Case Number'].apply(extract_date)

In [5]:
data['Clean Date'].value_counts()

0000/00/01    124
1950/00/00     13
1957/00/00     11
1942/00/00     11
1960/00/00     10
1958/00/00      9
1956/00/00      8
1898/00/00      8
1970/00/00      8
1941/00/00      7
1965/00/00      7
1954/00/00      7
1949/00/00      7
1959/00/00      7
1943/00/00      6
1955/00/00      6
1940/00/00      6
1995/07/28      5
2003/10/05      5
1952/00/00      5
1960/10/00      5
1929/00/00      5
1939/00/00      5
1938/00/00      5
1890/00/00      5
1956/08/00      5
2001/04/12      5
1970/01/23      4
1980/00/00      4
1926/00/00      4
             ... 
2002/08/05      1
1961/04/17      1
1961/04/14      1
1966/07/31      1
1909/07/15      1
1912/05/04      1
2004/06/02      1
2000/01/05      1
1971/10/23      1
1969/07/22      1
2010/12/11      1
2007/05/09      1
1932/02/16      1
1953/03/19      1
1977/03/17      1
1984/09/15      1
1889/07/08      1
1852/00/00      1
1886/04/26      1
1967/03/12      1
1968/02/04      1
1984/07/00      1
1984/07/01      1
1951/09/29      1
1986/09/22

In [6]:
month_count= 0
month_acumulated = 0

#Calculate the median month for the empty date months
for date in data['Clean Date']:
    if int(str(date).split("/")[1])>0:
        month_count +=1
        month_acumulated += int(date.split("/")[1])

med_month = month_acumulated/month_count
med_month_str = str(med_month)

# Add day 15 in the empty days and add month median to the empty months
def fix_bad_dates(date):
    date_splited = str(date).split("/")
    
    if len(date_splited)==3:
    
        if not int(date_splited[2])>0:
            date_splited[2]="15"

        if not int(date_splited[1])>0:
            date_splited[1]="0"+med_month_str

        else:
            return date_splited[0]+"/"+date_splited[1]+"/"+date_splited[2]
    else:
        return np.nan
    

In [7]:
data['Clean Date'] = data['Clean Date'].apply(fix_bad_dates)

In [8]:
data['Clean Date'].value_counts()

1956/08/15    8
2001/04/12    5
2003/10/05    5
1995/07/28    5
1960/10/15    5
1970/01/23    4
1906/10/10    4
1994/07/09    4
2010/01/09    4
2008/12/27    4
1993/08/15    4
1952/07/27    4
2003/07/15    4
2012/06/14    4
2014/12/28    4
2015/09/20    4
2013/06/25    3
2005/11/25    3
2007/09/30    3
2014/03/18    3
1965/05/15    3
2012/09/02    3
2003/06/24    3
1997/05/31    3
1934/04/01    3
1974/01/07    3
2011/06/06    3
2002/09/27    3
2014/06/09    3
2010/02/06    3
             ..
2010/12/17    1
1956/08/01    1
2007/05/09    1
2010/12/11    1
1969/07/22    1
1971/10/23    1
2000/01/05    1
2004/06/02    1
1909/07/15    1
1980/03/15    1
1961/04/15    1
1961/04/14    1
1961/04/17    1
1894/12/11    1
1951/09/29    1
1944/02/18    1
1932/02/16    1
1953/03/19    1
1977/03/17    1
1984/09/15    1
2000/10/09    1
1889/07/08    1
1886/04/26    1
1898/12/28    1
1967/03/12    1
1968/02/04    1
1852/05/24    1
1984/07/01    1
1936/12/19    1
1959/04/05    1
Name: Clean Date, dtype:

<h1>Time Cleaning</h1>

In [9]:
data['Time'].unique()

array([u'13h00', u'11h00', u'10h43', nan, u'15h15', u'14h30', u'15h40',
       u'Late afternoon', u'15h00', u'14h00', u'17h00', u'16h00', u'11h30',
       u'12h00', u'19h05', u'After noon', u'1300', u'10h00',
       u'14h30 / 15h30', u'22h00', u'16h20', u'14h34', u'15h25', u'14h55',
       u'06h00', u'17h30', u'08h30', u'Morning', u'Afternoon', u'15h45',
       u'18h00', u'17h46', u'Morning ', u'13h20', u'15h49', u'07h00',
       u'08h00', u'10h45', u'19h00', u'13h30', u'Midnight', u'12h30',
       u'09h30 / 10h00', u'10h30', u'18h15', u'04h00', u'14h50', u'13h50',
       u'19h20', u'10h25', u'16h50', u'10h45-11h15', u'16h45', u'15h52',
       u'07h30', u'06h15', u'14h40', u'19h45', u'17h10', u'15h30',
       u'10h10', u'08h45', u'Evening', u'09h00', u'12h10', u'18h05',
       u'19h30', u'11h41', u'12h25', u'09h30', u'17h51', u'16h12',
       u'12h45', u'10h50', u'13h45', u'14h15', u'16h30', u'09h45',
       u'11h45', u'10h20', u'18h30', u'06h45', u'05h00', u'03h30',
       u'Night', u

In [10]:
#Check if is a clean time
def is_clean_time(str_time):
    return len(str_time)==5 and str_time.split(":")[0] is not None

In [11]:
#Get the first time in the cases of a range of time
def get_first_time(str_time):
    regexph = re.compile(r'([0-2][0-9]h[0-5][0-9])')
    regexp = re.compile(r'([0-2][0-9]\:[0-5][0-9])')
    if regexph.search(str_time) is not None:
        str_time = regexph.findall(str_time)[0]
    elif regexp.search(str_time) is not None:
        str_time = regexp.findall(str_time)[0]
    
    return str_time

In [12]:
#Mapping the different text values to a time
def extract_time(time):
    str_time= str(time)
    str_time = get_first_time(str_time)
    
    if 'late afternoon' in str_time.lower():
        str_time = '20:00'
    if 'late afternon' in str_time.lower():
        str_time = '20:00'
    elif 'noon' in str_time.lower():
        str_time = '18:00'
    elif 'morning' in str_time.lower():
        str_time = '10:00'
    elif 'midnight' in str_time.lower():
        str_time = '00:00'
    elif 'night' in str_time.lower():
        str_time = '22:00'
    elif 'evening' in str_time.lower():
        str_time = '20:00'    
    elif 'after dark' in str_time.lower():
        str_time = '20:00'
    elif 'daybreak' in str_time.lower():
        str_time = '12:00'
    elif 'lunch' in str_time.lower():
        str_time = '09:00'
    
    str_time = str.replace(str_time,'h',':')
    str_time = str.replace(str_time,'j',':')
    
    if is_clean_time(str_time):
        return str_time+":00"
    else:
        return None #For Predict

    


In [13]:
data['Clean Time'] = data['Time'].apply(extract_time)

In [14]:
median=0
time_counter = 0
max_hour = 0
min_hour = 24
max_median = 0
min_median = 0
hours=0
hour=0
for time in data["Clean Time"]:
    if time is not None:
        time_counter +=1
        hour = int(time.split(":")[0])
        hours += hour
        if hour > max_hour:
            max_hour = int(hour)
        if hour < min_hour:
            min_hours = int(hour)
median = hours/time_counter


#Add median to empty values
def add_median_to_nan_values(time):
    if time == None:
        return str(median)+":00:00"
    else:
        return time


In [15]:
data['Clean Time'] = data['Clean Time'].apply(add_median_to_nan_values)

In [16]:
def check_time(time):
    splited_time = time.split(":")
    if len(splited_time)==3:
        if int(splited_time[0])<24 and int(splited_time[1])<60 and int(splited_time[2])<60:
            return time
    
    return np.nan
    

In [17]:
data['Clean Time'] = data['Clean Time'].apply(check_time)

In [18]:
data['Clean Time'].value_counts()

13:00:00    3374
18:00:00     249
10:00:00     179
11:00:00     126
12:00:00     112
15:00:00     102
14:00:00      95
16:00:00      94
20:00:00      84
16:30:00      73
14:30:00      72
17:30:00      70
17:00:00      68
22:00:00      65
15:30:00      62
11:30:00      61
13:30:00      56
09:00:00      51
10:30:00      47
09:30:00      33
12:30:00      33
07:30:00      31
08:00:00      30
18:30:00      29
08:30:00      25
19:00:00      25
07:00:00      22
17:15:00      18
14:45:00      17
15:45:00      17
            ... 
12:54:00       1
08:55:00       1
06:10:00       1
00:30:00       1
11:06:00       1
19:35:00       1
18:25:00       1
22:30:00       1
08:57:00       1
13:10:00       1
19:55:00       1
13:14:00       1
12:02:00       1
02:45:00       1
15:19:00       1
23:30:00       1
17:17:00       1
16:12:00       1
11:51:00       1
12:46:00       1
17:11:00       1
09:11:00       1
13:42:00       1
01:50:00       1
09:05:00       1
15:52:00       1
09:20:00       1
03:10:00      

In [19]:
data['Clean Time'].unique()

array(['13:00:00', '11:00:00', '10:43:00', '15:15:00', '14:30:00',
       '15:40:00', '20:00:00', '15:00:00', '14:00:00', '17:00:00',
       '16:00:00', '11:30:00', '12:00:00', '19:05:00', '18:00:00',
       '10:00:00', '22:00:00', '16:20:00', '14:34:00', '15:25:00',
       '14:55:00', '06:00:00', '17:30:00', '08:30:00', '15:45:00',
       '17:46:00', '13:20:00', '15:49:00', '07:00:00', '08:00:00',
       '10:45:00', '19:00:00', '13:30:00', '00:00:00', '12:30:00',
       '09:30:00', '10:30:00', '18:15:00', '04:00:00', '14:50:00',
       '13:50:00', '19:20:00', '10:25:00', '16:50:00', '16:45:00',
       '15:52:00', '07:30:00', '06:15:00', '14:40:00', '19:45:00',
       '17:10:00', '15:30:00', '10:10:00', '08:45:00', '09:00:00',
       '12:10:00', '18:05:00', '19:30:00', '11:41:00', '12:25:00',
       '17:51:00', '16:12:00', '12:45:00', '10:50:00', '13:45:00',
       '14:15:00', '16:30:00', '09:45:00', '11:45:00', '10:20:00',
       '18:30:00', '06:45:00', '05:00:00', '03:30:00', '06:30:

In [20]:
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
Clean Date                object
Clean Time                object
dtype: object

<h1>Hours</h1>

In [21]:
def get_hour(time):
    return int(time.split(":")[0])

In [22]:
data['Hour'] = data['Clean Time'].apply(get_hour)

<h1>Sex</h1>

In [23]:
data["Sex "].value_counts()

M      4835
F       585
M         2
N         1
.         1
lli       1
Name: Sex , dtype: int64

In [24]:
def fix_sex(sex):
    sex = str(sex)
    if "M" in sex:
        return 0
    if "F" in sex:
        return 1
    return 0 #default 0 because is the most frequent
        
     

In [25]:
data["Sex Clean"] = data["Sex "].apply(fix_sex)

In [26]:
data["Sex Clean"].value_counts()

0    5407
1     585
Name: Sex Clean, dtype: int64

<h1>Join Date</h1>

In [27]:
def string_to_datetime(complet_date):
    #return pd.to_datetime(complet_date, format='%H/%m/%s', errors='ignore')
    #return np.datetime64(complet_date)
    
    return pd.to_datetime(complet_date, format='%dd/%BB/%YYYY %H:%m:%s', errors='coerce')

In [28]:

complete_date = data["Clean Date"].map(str)+" "+ data["Clean Time"]
data["DateTime"] = complete_date.apply(string_to_datetime)
data.dtypes



Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
Clean Date                object
Clean Time                object
Hour                       int64
Sex Clean                  int64
DateTime                  object
dtype: object

In [29]:
data["DateTime"].value_counts()

None 13:00:00          491
None 18:00:00           10
1956/08/15 13:00:00      7
None 10:00:00            5
1906/10/10 13:00:00      4
1960/10/15 13:00:00      4
1970/01/23 13:00:00      4
1952/07/27 13:00:00      4
1965/05/15 13:00:00      3
1999/12/31 13:00:00      3
2012/06/14 18:00:00      3
1916/07/12 13:00:00      3
2000/07/15 13:00:00      3
1974/08/16 13:00:00      3
1964/07/15 13:00:00      3
1996/02/10 13:00:00      3
1960/11/15 13:00:00      3
1886/08/15 13:00:00      3
1962/06/11 22:00:00      3
None 12:00:00            3
1977/03/13 13:00:00      3
2003/07/15 13:00:00      3
1960/04/15 13:00:00      3
1999/11/15 13:00:00      3
1885/07/26 13:00:00      3
None 22:00:00            2
1993/08/15 13:00:00      2
1959/04/09 13:00:00      2
1953/04/15 13:00:00      2
1887/02/08 13:00:00      2
                      ... 
2004/05/28 11:00:00      1
2010/02/13 13:30:00      1
2006/07/28 16:30:00      1
2011/06/19 13:00:00      1
1981/12/19 20:00:00      1
1965/03/23 11:30:00      1
2

<h1>Get Month</h1>

In [65]:
#If the month is empty put random month
def get_month(date):
    array = date.split("/")
    if len(array)>1:
        if "00" in array[1]:
            return randint(1,12)
        if len(array[1])<2 and "0" in array[1]:
            return randint(1,12)
        
        return int(array[1])
    else:
        return randint(1,12)


data['Month'] = data['DateTime'].map(get_month)

In [66]:
data["Month"].unique()

array([ 9,  8,  7,  6,  5,  4,  3,  2,  1, 12, 11, 10])

In [67]:
data["Month"].value_counts()

7     683
8     624
9     579
1     549
6     515
12    459
4     453
3     443
10    443
11    440
5     405
2     399
Name: Month, dtype: int64

<h1>Clean Type Attack</h1>
Only put the boating, provoked or unprovoked, because we can not predict a sea desaster.

In [33]:
data['Type'].unique()

array([u'Unprovoked', u'Boat', u'Provoked', u'Invalid', u'Sea Disaster',
       u'Boating'], dtype=object)

In [68]:
data["Type"].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: Type, dtype: int64

In [34]:
def mapping_type_to_identifier(attack_type):
    if "Unprovoked" in attack_type:
        return 1
    elif "Boat" in attack_type:
        return 2
    elif "Invalid" in attack_type:
        return 1
    elif "Sea Disaster" in attack_type:
        return 1
    elif "Provoked" in attack_type:
        return 3
    elif "Boating" in attack_type:
        return 2
    else:
        return 1

In [35]:
data['Id Type Attack'] = data['Type'].apply(mapping_type_to_identifier)

<h1>Clean Age</h1>

In [37]:
data["Age"].unique()

array([u'16', u'36', u'43', nan, u'60s', u'51', u'50', u'12', u'9', u'22',
       u'25', u'37', u'20', u'49', u'15', u'21', u'40', u'72', u'18',
       u'29', u'31', u'11', u'10', u'59', u'42', u'34', u'35', u'19', u'6',
       u'27', u'64', u'60', u'23', u'52', u'13', u'57', u'48', u'39',
       u'24', u'26', u'69', u'46', u'Teen', u'41', u'45', u'65', u'38',
       u'71', u'32', u'58', u'28', u'54', u'44', u'14', u'7', u'62',
       u'40s', u'68', u'47', u'17', u'30', u'63', u'70', u'18 months',
       u'53', u'20s', u'33', u'30s', u'50s', u'8', u'61', u'55', u'teen',
       u'66', u'77', u'74', u'3', u'56', u'28 & 26', u'5', u'86',
       u'18 or 20', u'12 or 13', u'46 & 34', u'28, 23 & 30', u'Teens',
       u'36 & 26', u'8 or 10', u'84', u'\xa0 ', u' ', u'30 or 36',
       u'6\xbd', u'21 & ?', u'75', u'33 or 37', u'mid-30s', u'73',
       u'23 & 20', u' 30', u'7      &    31', u' 28', u'20?', u"60's",
       u'32 & 30', u'16 to 18', u'87', u'67', u'Elderly', u'mid-20s',
       u'Ca

In [38]:
#Mapping text values to integer value and get the firs value if has a age rang
def clean_age(age):
    age = str(age)
    clean_age = str(age)
    
    if "teen" in age.lower():
        clean_age = "10"
        
    if "adult" in age.lower():
        clean_age = "18"
        
    if "young" in age.lower():
        clean_age = "14"
    
    if "middle-age" in age.lower():
        clean_age = "35"
    
    
    regexp = re.compile(r'([0-9]*[0-9])')
    if regexp.search(clean_age) is not None:
        clean_age = regexp.findall(clean_age)[0]
    
    for char in age:
        if char.isalpha():
            str.replace(clean_age,char,'')
    if clean_age.isdigit():
        return long(clean_age)
    else:
        return -1 #For Predict
    

In [39]:
data["Clean Age"] = data["Age"].apply(clean_age)

In [40]:
age_acumulate = 0
age_counter = 0
for age in data["Clean Age"]:
    if age is not -1:
        age_acumulate += age
        age_counter +=1
        
age_median = age_acumulate/age_counter

#For empty values add age median
def add_age_median(age):
    if age == -1:
        return age_median
    else:
        return age

In [41]:
data["Clean Age"].value_counts()

-1     2692
 18     150
 17     150
 20     144
 19     139
 15     135
 16     135
 21     118
 22     113
 24     103
 25     103
 14     101
 13      93
 30      84
 23      82
 26      80
 28      79
 27      78
 29      75
 12      71
 10      66
 32      65
 35      65
 40      52
 31      51
 38      47
 34      46
 33      44
 36      43
 43      43
       ... 
 54      14
 53      13
 58      12
 61      11
 59      11
 63       9
 56       9
 69       8
 62       8
 5        7
 70       6
 68       6
 64       4
 65       4
 3        4
 71       3
 66       3
 77       3
 74       2
 1        2
 75       2
 2        2
 78       2
 73       1
 67       1
 84       1
 86       1
 72       1
 81       1
 87       1
Name: Clean Age, dtype: int64

In [42]:

data["Clean Age"] = data["Clean Age"].apply(add_age_median)

In [43]:
data["Clean Age"].unique()

array([16, 36, 43, 14, 60, 51, 50, 12,  9, 22, 25, 37, 20, 49, 15, 21, 40,
       72, 18, 29, 31, 11, 10, 59, 42, 34, 35, 19,  6, 27, 64, 23, 52, 13,
       57, 48, 39, 24, 26, 69, 46, 41, 45, 65, 38, 71, 32, 58, 28, 54, 44,
        7, 62, 68, 47, 17, 30, 63, 70, 53, 33,  8, 61, 55, 66, 77, 74,  3,
       56,  5, 86, 84, 75, 73, 87, 67,  1,  2, 81, 78])

In [44]:
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
Clean Date                object
Clean Time                object
Hour                       int64
Sex Clean                  int64
DateTime                  object
Month                      int64
Id Type Attack             int64
Clean Age                  int64
dtype: object

<h1>Clean Area</h1>

In [45]:
data["Country"].value_counts()

USA                          2116
AUSTRALIA                    1279
SOUTH AFRICA                  565
PAPUA NEW GUINEA              133
NEW ZEALAND                   125
BRAZIL                        102
BAHAMAS                        98
MEXICO                         81
ITALY                          71
FIJI                           62
PHILIPPINES                    59
REUNION                        57
NEW CALEDONIA                  51
MOZAMBIQUE                     44
CUBA                           42
SPAIN                          40
INDIA                          37
EGYPT                          36
CROATIA                        34
JAPAN                          32
PANAMA                         32
SOLOMON ISLANDS                29
IRAN                           29
GREECE                         25
HONG KONG                      24
JAMAICA                        23
FRENCH POLYNESIA               22
INDONESIA                      20
ENGLAND                        19
PACIFIC OCEAN 

In [69]:
#Clean some tips, for the dobule countires we take the first
def clean_countries(country):
    
    country = str(country).upper().replace("?","")
    
    if len(country.split("/"))>1:
        return country.split("/")[0]
    else:
        return country
    
data["Country"] = data["Country"].apply(clean_countries) 
data["Country"].value_counts()

USA                         2116
AUSTRALIA                   1279
SOUTH AFRICA                 565
PAPUA NEW GUINEA             133
NEW ZEALAND                  125
BRAZIL                       102
BAHAMAS                       98
MEXICO                        81
ITALY                         71
FIJI                          65
PHILIPPINES                   59
REUNION                       57
NEW CALEDONIA                 51
MOZAMBIQUE                    44
NAN                           43
CUBA                          42
SPAIN                         40
INDIA                         37
EGYPT                         36
CROATIA                       34
JAPAN                         32
PANAMA                        32
IRAN                          29
SOLOMON ISLANDS               29
GREECE                        25
HONG KONG                     24
JAMAICA                       23
FRENCH POLYNESIA              22
INDONESIA                     20
ENGLAND                       19
          

In [47]:
countries = data["Country"].unique()
countries_list =[]
for country in countries:
    countries_list.append(country)
    
#Mapping countires to identifier
def id_countries(country):
    return countries_list.index(country)


In [48]:
data["Country Id"] = data["Country"].apply(id_countries)
data["Country Id"].value_counts()

0      2116
1      1279
9       565
33      133
11      125
16      102
4        98
25       81
24       71
15       65
44       59
3        57
2        51
21       44
29       43
70       42
5        40
88       37
10       36
49       34
7        32
67       32
101      29
62       29
27       25
96       24
39       23
13       22
12       20
71       19
       ... 
183       1
187       1
174       1
170       1
162       1
154       1
109       1
125       1
129       1
133       1
137       1
141       1
145       1
153       1
157       1
173       1
177       1
181       1
185       1
189       1
191       1
30        1
78        1
82        1
98        1
106       1
126       1
138       1
146       1
193       1
Name: Country Id, dtype: int64

<h1>Activity</h1>

In [50]:
data["Activity"].value_counts()

Surfing                                                                                              904
Swimming                                                                                             819
Fishing                                                                                              414
Spearfishing                                                                                         321
Bathing                                                                                              153
Wading                                                                                               144
Diving                                                                                               123
Standing                                                                                              97
Snorkeling                                                                                            77
Scuba diving                                           

In [70]:
#Mapping the most frequents activities
def mapping_activities(activity):
    activity = str(activity).upper()
    
    if "SURFING" in activity:
        return 0
    elif "SWIMMING" in activity:
        return 1
    elif "FISHING" in activity:
        return 2
    elif "SPEARFISHING" in activity:
        return 3
    elif "BATHING" in activity:
        return 4
    elif "WADING" in activity:
        return 5
    elif "DIVING" in activity:
        return 6
    elif "STANDING" in activity:
        return 7
    elif "SNORKELING" in activity:
        return 8
    elif "SCUBA DIVING" in activity:
        return 9
    elif "BODY BOARDING" in activity:
        return 10
    elif "TREADING WATER" in activity:
        return 11
    elif "PEARL DIVING" in activity:
        return 12
    elif "BOOGIE BOARDING" in activity:
        return 13
    elif "KAYAKING" in activity:
        return 14
    elif "FREE DIVING" in activity:
        return 15
    elif "FELL OVERBOARD" in activity:
        return 16
    elif "WINFSURFING" in activity:
        return 17
    else:
        return 18
    

In [52]:
data["Activity Id"] = data["Activity"].apply(mapping_activities)

In [53]:
data["Activity Id"].value_counts()

18    1489
2     1122
1     1056
0     1054
6      509
4      180
5      160
7      128
8       82
10      58
13      45
16      41
11      37
14      31
Name: Activity Id, dtype: int64

<h1>Fatal</h1>

In [54]:
data["Fatal (Y/N)"].value_counts()

N                 4315
Y                 1552
UNKNOWN             94
 N                   8
N                    1
--M524--M3133Y       1
n                    1
F                    1
Name: Fatal (Y/N), dtype: int64

In [55]:
def clean_fatal(fatal):
    fatal = str(fatal).upper()
    if "UNKNOWN" in fatal:
        return 1
    if "--M524--M3133Y" in fatal:
        return 1
    if "N" in fatal:
        return 1
    if "Y" in fatal:
        return 2
    return 1

In [56]:
data["Clean Fatal"] = data["Fatal (Y/N)"].apply(clean_fatal)

In [57]:
data["Clean Fatal"].unique()

array([1, 2])

In [58]:
data["Clean Fatal"].value_counts()

1    4440
2    1552
Name: Clean Fatal, dtype: int64

<H1>Make CSV file with the cleaned and mapped data</H1>

In [71]:
clean_data = pd.DataFrame()

In [72]:
clean_data["Clean Fatal"] = data["Clean Fatal"]
clean_data["Activity Id"] = data["Activity Id"]
clean_data["Country Id"] = data["Country Id"]
clean_data["Clean Age"] = data["Clean Age"]
clean_data["Month"] = data["Month"]
clean_data['Id Type Attack'] = data['Id Type Attack']
clean_data['Hour'] = data['Hour']
clean_data["Sex Clean"] = data["Sex Clean"]

In [61]:
clean_data.to_csv("output.csv", sep=',', encoding='utf-8')